# NEAT Algorithm

### Making NeuroEvolution of Augmenting Topologies (NEAT)

NEAT is a genetic algorithm which as the name suggests utilizes an evolutionary approach to machine learning. In essence, it works by setting up a population of the simplest possible networks for the given task (e.g. if the task provided three inputs and expected a single output, we would create networks with 3 input neurons and 1 output neuron with no connections). The networks are then tested on the given task/problem and those who perform best according to given criteria go on to multiply and breed with one another. Thus NEAT is an unsupervised AI, although at least this particular version is not general, since we have to adjust the structure for any given problem.

I hope to make the algorithm support as many methods as possible while being as simple as possible and would appriciate any input to make this happen.

In [1]:
# Relenvant imports
import math
import random

### Simple Neuron, Synapse and Network (for NEAT use)

We need to create our NN class such that we can customize the starting parameters (inputs and outputs), while it has the capability to mutate itself. Another important feature to include is the innovation number of synapses such that we can identify similar neuron connections across networks, which allows us to divide our network population into species, thereby optimizing the crossover by identifying fit species (not only individuals). 

This is because an individual might get lucky in one iteration but if a majority of networks with a specific connection performs well and a majority without set connection performs worse, a non-insignificant degree of belief can be attributed to the importance og this connection (especially if the networks are simple).

### Additional notes:

As was noted in "NN Basics" the error margin seemed to decease by having the majority of hidden neurons in the earlier layers (simplified explanation). This principle will be implemented by placing all new neurons in the first hidden layer.

Have included some options in these classes that aren't yet implimented in the NEAT algorithm.

In [2]:
class Neuron:
    def __init__(self, place_in_network):
        self.no = place_in_network  # represented as [layer, index_in_layer]. Especially useful when keeping track of innovation connections
        self.connections = []  # connected forward neurons, useful 
        self.value = 0  # accumulated inputs (prior to activation)
        self.activated_value = 0  # value post activation function
        self.enabled = True

    def sigmoid_activation(self):
        self.activated_value = 1 / (1 + math.exp(-self.value))

In [3]:
class Synapse:
    def __init__(self, inno_no, from_neuron, to_neuron, weight=random.uniform(-2, 2)):
        self.inno_no = inno_no  # innovation number to keep track when performing crossover
        self.from_neuron = from_neuron
        self.to_neuron = to_neuron
        self.weight = weight
        self.enabled = True

In [4]:
class NeuralNet:
    def __init__(self):
        self.input = []
        self.output = []
        self.hidden = []
        self.fitness = 0

        self.neurons = []  # neurons
        self.synapses = []  # synapses
        self.layers = []  # layers

    def generate_empty_genome(self, input_size, output_size):
        self.neurons = []
        self.synapses = []
        self.layers = []
        self.hidden = []
        self.fitness = 0

        self.input = [Neuron(['input', i]) for i in range(input_size)]
        self.output = [Neuron(['output', i]) for i in range(output_size)]

        self.neurons += self.input + self.output
        self.layers.append(self.input)
        self.layers.append(self.output)

    def generate_synapse(self, inno_no, from_neuron, to_neuron, weight=random.uniform(-2, 2)):
        from_neuron.connections.append(to_neuron)

        connection = Synapse(inno_no, from_neuron, to_neuron, weight)
        self.synapses.append(connection)

    def mutate_synapse(self, shift=True):
        if len(self.synapses) != 0:
            synapse = self.synapses[random.randint(0, len(self.synapses) - 1)]

            # 2 possibilities: a randomized (new) or adjusted (shifted) value
            if shift:
                synapse.weight = random.uniform(0, 2)
            else:
                synapse.weight * random.uniform(-2, 2)

    def enable_disable_synapse(self):
        if len(self.synapses) != 0:
            synapse = self.synapses[random.randint(0, len(self.synapses) - 1)]
            if synapse.enabled:
                synapse.enabled = False
            elif not synapse.enabled:
                synapse.enabled = True

    def generate_new_neuron(self):
        neuron = Neuron(['hidden', len(self.hidden)])
        self.hidden.append(neuron)
        self.neurons.append(neuron)

    def enable_disable_neuron(self):
        if len(self.hidden) != 0:
            neuron = self.hidden[random.randint(0, len(self.hidden) - 1)]
            if neuron.enabled:
                neuron.enabled = False
            elif not neuron.enabled:
                neuron.enabled = True

### NEAT Algorithm

Creates a population of neural nets, then over generations mutate and breed them. It includes several different possible ways of mutating, and hopefully I'll be able to add more as my knowledge expands. Likewise it includes different ways of breeding.

### Possible mutations:
<ul>
  <li>Create a random connection (synapse)</li>
  <li>Randomize/shift the value of a random enabled connection (synapse) - two ways</li>
  <li>Disable/enable a random connection (synapse)</li>
  <li>Create a random new neuron with two connections (synapses)</li>
</ul>

### Possible ways of breeding:
<ul>
  <li>Cloning</li>
  <li>Interspecies breeding</li>
  <li>Crossover breeding</li>
</ul>

In [5]:
class NEAT:
    def __init__(self, max_gen=None):
        self.population = []  # a list of all the current networks
        self.species = {}  # a dictionary where the keys are strings of a list of inno_nums indicating the species and where the values are networks belonging to that species
        self.inno_connections = []
        self.size = 0
        self.current_gen = 1
        self.max_gen = max_gen

    def generate_new_population(self, size):
        self.current_gen = 1

        self.population = [NeuralNet() for _ in range(size)]
        self.size = size

    def perform_mutation(self):
        for network in self.population:

            # 10% chance to spawn a random synapse
            if random.randint(0, 9) == 0:
                random_neuron_1 = network.neurons[random.randint(0, len(network.neurons) - 1)]
                random_neuron_2 = None
                while random_neuron_2 is None:
                    current_neuron = network.neurons[random.randint(0, len(network.neurons) - 1)]
                    if current_neuron.no[0] == random_neuron_1.no[0] and current_neuron.no[0] != 'hidden':
                        continue
                    else:
                        random_neuron_2 = current_neuron

                if random_neuron_1.no[0] == 'output' or random_neuron_2.no[0] == 'input':
                    if [random_neuron_2.no, random_neuron_1.no] not in self.inno_connections:
                        self.inno_connections.append([random_neuron_2.no, random_neuron_1.no])

                    inno_num = self.inno_connections.index([random_neuron_2.no, random_neuron_1.no])
                    if inno_num not in [i.inno_no for i in network.synapses]:
                        network.generate_synapse(inno_num, random_neuron_2, random_neuron_1, random.uniform(-2, 2))
                        print("Synapse generated in network {} between {} and {}.".format(str(self.population.index(network)), str(random_neuron_2.no), str(random_neuron_1.no)))

                    else:  # if there's already such a connection, a mutation will happen instead
                        network.mutate_synapse()

                else:
                    if [random_neuron_1.no, random_neuron_2.no] not in self.inno_connections:
                        self.inno_connections.append([random_neuron_1.no, random_neuron_2.no])
                    inno_num = self.inno_connections.index([random_neuron_1.no, random_neuron_2.no])
                    if inno_num not in [i.inno_no for i in network.synapses]:
                        network.generate_synapse(inno_num, random_neuron_1, random_neuron_2, random.uniform(-2, 2))
                        print("Synapse generated in network {} between {} and {}.".format(str(self.population.index(network)), str(random_neuron_1.no), str(random_neuron_2.no)))

                    else:  # if there's already such a connection, a mutation will happen instead
                        network.mutate_synapse()

            # 5% chance to mutate a random synapse (if one exists)
            elif random.randint(0, 19) == 0:
                network.mutate_synapse()

            # 5% chance to enable/disable a random synapse (if one exists)
            elif random.randint(0, 19) == 0:
                network.enable_disable_synapse()

            # 5% chance to spawn a randomly connected new neuron
            elif random.randint(0, 19) == 0:
                new_neuron = Neuron(['hidden', len(network.hidden)])
                random_neuron_1 = network.neurons[random.randint(0, len(network.neurons) - 1)]
                random_neuron_2 = None
                while random_neuron_2 is None:
                    current_neuron = network.neurons[random.randint(0, len(network.neurons) - 1)]
                    if current_neuron.no[0] == random_neuron_1.no[0] and current_neuron.no[0] != 'hidden':
                        continue
                    else:
                        random_neuron_2 = current_neuron

                if random_neuron_1.no[0] == 'output' or random_neuron_2.no[0] == 'input':
                    if [random_neuron_2.no, new_neuron.no] not in self.inno_connections:
                        self.inno_connections.append([random_neuron_2.no, new_neuron.no])
                    if [new_neuron.no, random_neuron_1.no] not in self.inno_connections:
                        self.inno_connections.append([new_neuron.no, random_neuron_1.no])

                    inno_num_1 = self.inno_connections.index([random_neuron_2.no, new_neuron.no])
                    inno_num_2 = self.inno_connections.index([new_neuron.no, random_neuron_1.no])

                    network.generate_synapse(inno_num_1, random_neuron_2, new_neuron, 2)
                    network.generate_synapse(inno_num_2, new_neuron, random_neuron_1, random.uniform(-2, 2))

                    print("Neuron generated in network {} with connections between {} and {}.".format(str(self.population.index(network)), str(random_neuron_2.no), str(random_neuron_1.no)))

                else:
                    if [random_neuron_1.no, new_neuron.no] not in self.inno_connections:
                        self.inno_connections.append([random_neuron_1.no, new_neuron.no])
                    if [new_neuron.no, random_neuron_2.no] not in self.inno_connections:
                        self.inno_connections.append([new_neuron.no, random_neuron_2.no])

                    inno_num_1 = self.inno_connections.index([random_neuron_1.no, new_neuron.no])
                    inno_num_2 = self.inno_connections.index([new_neuron.no, random_neuron_2.no])

                    network.generate_synapse(inno_num_1, random_neuron_1, new_neuron, 2)
                    network.generate_synapse(inno_num_2, new_neuron, random_neuron_2, random.uniform(-2, 2))

                    print("Neuron generated in network {} with connections between {} and {}.".format(str(self.population.index(network)), str(random_neuron_1.no), str(random_neuron_2.no)))

                network.hidden.append(new_neuron)
                network.neurons.append(new_neuron)

    # makes species, then sorts the species and removes the least fit networks, then breeds and mutates new generation
    def perform_crossover(self):

        self.perform_speciation()

        for networks in self.species.values():
            if len(networks) == 1:
                continue
            else:
                networks.sort(key=lambda x: x.fitness, reverse=True)
                survivors = len(networks) // 2 + 1
                while len(networks) != survivors:
                    network_to_remove = networks[-1]
                    networks.remove(network_to_remove)
                    self.population.remove(network_to_remove)

        print("Population size after selection: {}".format(len(self.population)))
        print("Cloning, breeding and repopulating...")

        self.perform_breeding()

        self.current_gen += 1

        self.perform_mutation()

    # divides self.population into self.species based on their network structure (does not sort for fitness)
    def perform_speciation(self):
        self.species = {}  # last generation's species cleared
        for network in self.population:
            gene = []
            for synapse in network.synapses:
                gene.append(synapse.inno_no)
            gene.sort()
            if str(gene) in self.species:
                self.species[str(gene)].append(network)
            else:
                self.species.setdefault(str(gene), [network])

    # based on species and the individual networks within these, different types of breeding is performed
    def perform_breeding(self):
        species_fitness = []
        for species, networks in self.species.items():
            if len(networks) >= 3:
                fit = (networks[0].fitness + networks[1].fitness + networks[2].fitness) // 3
                species_fitness.append([species, fit])
            else:
                species_fitness.append([species, networks[0].fitness])

        species_fitness.sort(key=lambda x: x[1], reverse=True)

        while len(self.population) != self.size:
            chance = random.randint(0, 9)
            parent_1 = None
            parent_2 = None

            same_species = False

            # 40% chance for to clone top network in a species
            if chance <= 3:
                while parent_1 is None:
                    for i in range(len(self.species)):
                        if random.randint(0, 1) == 0:
                            parent_1 = self.species[species_fitness[i][0]][0]
                            break

            # 40% chance for top species to breed with itself
            elif chance <= 7 and len(self.species[species_fitness[0][0]]) >= 2:
                parent_1 = self.species[species_fitness[0][0]][0]
                parent_2 = self.species[species_fitness[0][0]][1]
                same_species = True

            # 20% chance for top to species to breed with each other
            elif chance <= 9:
                parent_1 = self.species[species_fitness[0][0]][0]
                parent_2 = self.species[species_fitness[1][0]][0]

            # cloning
            new_network = NeuralNet()
            new_network.generate_empty_genome(len(parent_1.input), len(parent_1.output))
            num_of_new_hidden = len(parent_1.hidden) - len(new_network.hidden)
            while num_of_new_hidden > 0:
                new_network.generate_new_neuron()
                num_of_new_hidden -= 1

            for i, synapse in enumerate(parent_1.synapses):
                inno_num = synapse.inno_no

                going_from = new_network.neurons[parent_1.neurons.index(synapse.from_neuron)]
                going_to = new_network.neurons[parent_1.neurons.index(synapse.to_neuron)]
                if same_species:
                    w = random.uniform(synapse.weight, parent_2.synapses[i].weight)
                else:
                    w = synapse.weight
                new_network.synapses.append(Synapse(inno_num, going_from, going_to, w))
                going_from.connections.append(going_to)

            # breeding
            if parent_2 is not None and not same_species:

                num_of_new_hidden = len(parent_2.hidden) - len(new_network.hidden)
                while num_of_new_hidden > 0:
                    new_network.generate_new_neuron()
                    num_of_new_hidden -= 1

                for i, synapse in enumerate(parent_2.synapses):
                    inno_num = synapse.inno_no
                    going_from = new_network.neurons[parent_2.neurons.index(synapse.from_neuron)]
                    going_to = new_network.neurons[parent_2.neurons.index(synapse.to_neuron)]
                    w = synapse.weight
                    if going_to not in going_from.connections:
                        new_network.synapses.append(Synapse(inno_num, going_from, going_to, w))
                        going_from.connections.append(going_to)

            self.population.append(new_network)

Alright, let's instantiate NEAT with a population of 100 networks. Then we need to give those networks a structure (we will for the purpose of simplicity have 3 inputs and 1 output). Finally, let's perform a mutation to see what kind of changes we get in our population.

In [6]:
neat = NEAT()
neat.generate_new_population(100)

for network in neat.population:
    network.generate_empty_genome(3, 1)

neat.perform_mutation()

Synapse generated in network 1 between ['input', 1] and ['output', 0].
Synapse generated in network 4 between ['input', 0] and ['output', 0].
Synapse generated in network 7 between ['input', 1] and ['output', 0].
Neuron generated in network 8 with connections between ['input', 2] and ['output', 0].
Synapse generated in network 9 between ['input', 1] and ['output', 0].
Synapse generated in network 11 between ['input', 1] and ['output', 0].
Synapse generated in network 28 between ['input', 2] and ['output', 0].
Neuron generated in network 35 with connections between ['input', 2] and ['output', 0].
Neuron generated in network 47 with connections between ['input', 2] and ['output', 0].
Synapse generated in network 51 between ['input', 2] and ['output', 0].
Neuron generated in network 55 with connections between ['input', 0] and ['output', 0].
Neuron generated in network 67 with connections between ['input', 2] and ['output', 0].
Synapse generated in network 68 between ['input', 1] and ['ou

As we can see about 15-20% of the population makes some sort of mutation. This of course means that about 80% still doesn't have any connections at all and would take no action in any environment. As such it might be useful to give the first generation a guaranteed (100%) mutation, but for now let's stick with this. The statements of the mutation step makes it very easy to visualize the structure of our networks. For example (in this population) network 1 and 8 look as follows:

Network 1:
<br>0 
<br>1 ---- 0 
<br>2

Network 8:
<br>0 
<br>1
<br>2 ---- 0 ---- 0

Let's try to give our network population some random fitness levels (normally the networks will obtain these based on their performance in a given environment). Then we will perform the crossover which kills off part of the population based of their fitness and species (fit individuals survive and unfit ones are killed although individuals that are rare can survive even though the have a bad fitness score). Then cloning and breeding is performed on our species relative to their fitness level. Finally after a new generation is produced a mutation step occurs.

In [7]:
for network in neat.population:
    network.fitness = random.randint(0, 99)

neat.perform_crossover()

Population size after selection: 54
Cloning, breeding and repopulating...
Synapse generated in network 15 between ['input', 1] and ['output', 0].
Synapse generated in network 20 between ['input', 2] and ['output', 0].
Synapse generated in network 32 between ['input', 2] and ['output', 0].
Synapse generated in network 34 between ['input', 1] and ['output', 0].
Synapse generated in network 57 between ['input', 2] and ['output', 0].
Synapse generated in network 65 between ['input', 1] and ['output', 0].
Neuron generated in network 77 with connections between ['input', 0] and ['output', 0].
Synapse generated in network 91 between ['input', 1] and ['output', 0].
Synapse generated in network 97 between ['input', 0] and ['output', 0].


In [8]:
print(neat.current_gen)

2


And we have our second generation ready to go.